In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [10]:
# Extract main data frame and calculate vibration error scores

# Initialize an empty DataFrame to hold all combined data
combined_data = pd.DataFrame()

# Iterate through each participant directory and load the corresponding file
for i in range(1, 16):
    file_path = f'../data/p{i}/p{i}_main.csv'  # Constructs the file name
    
    # Load the data
    temp_df = pd.read_csv(file_path)
    
    # Add a column for participant_id
    temp_df['participant_id'] = i
    # temp_df = temp_df[temp_df['terminal_feedback'] != 1]
    temp_df = temp_df[(temp_df["move_times"] > 0.4) & (temp_df['mean_velocity'] > 20)]
    # Append to the combined DataFrame
    combined_data = pd.concat([combined_data, temp_df], ignore_index=True)

# Now 'combined_data' contains data from all participants, with an identifier column
combined_data.to_csv('../all_data_vision.csv')

combined_data = combined_data[combined_data['terminal_feedback'] == 0]
mean_scores_by_participant_and_vibration = combined_data.groupby(['participant_id', 'vibration'])[['elbow_end_deg', 'error', 'move_times', 'mean_velocity']].mean().reset_index()
baseline_scores = mean_scores_by_participant_and_vibration[mean_scores_by_participant_and_vibration['vibration'] == 0]

baseline_scores = baseline_scores.rename(columns={
    'elbow_end_deg': 'baseline_elbow_end_deg',
    'error': 'baseline_error',
    'move_times': 'baseline_move_times'
})
baseline_scores['vibration'] = 0 

comparison_df = mean_scores_by_participant_and_vibration.merge(
    baseline_scores.drop('vibration', axis=1),
    on='participant_id',
    suffixes=('', '_baseline')
)

comparison_df['elbow_end_deg_diff'] = comparison_df['elbow_end_deg'] - comparison_df['baseline_elbow_end_deg']
comparison_df['error_diff'] = comparison_df['error'] - comparison_df['baseline_error']
comparison_df['move_times_diff'] = comparison_df['move_times'] - comparison_df['baseline_move_times']

comparison_df['vibration'] = comparison_df['vibration'].replace({
    0: 'No Vibration',
    1: 'Dual',
    2: 'Triceps',
    3: 'Biceps'
})


comparison_df.to_csv('../summary_data.csv')


In [4]:
# Initialize an empty DataFrame to hold all combined data
baseline_data = pd.DataFrame()
post_data = pd.DataFrame()

# Iterate through each participant directory and load the corresponding file
for i in range(1, 16):
    file_path1 = f'../../data/p{i}/p{i}_baseline.csv'  # Constructs the file name
    file_path2 = f'../../data/p{i}/p{i}_post.csv'  # Constructs the file name
    
    # Load the data
    temp_df1 = pd.read_csv(file_path1)
    temp_df2 = pd.read_csv(file_path2)
    
    # Add a column for participant_id
    temp_df1['participant_id'] = i
    temp_df = temp_df1[temp_df1['terminal_feedback'] != 1]
    temp_df1 = temp_df1[(temp_df1["move_times"] > 0.4) & (temp_df1['mean_velocity'] > 20)]
    # Append to the combined DataFrame
    baseline_data = pd.concat([baseline_data, temp_df1], ignore_index=True)

    baseline_data.to_csv('../../baseline_data.csv')

    # Add a column for participant_id
    temp_df2['participant_id'] = i
    temp_df = temp_df2[temp_df2['terminal_feedback'] != 1]
    temp_df2 = temp_df2[(temp_df2["move_times"] > 0.4) & (temp_df2['mean_velocity'] > 20)]
    # Append to the combined DataFrame
    post_data = pd.concat([post_data, temp_df2], ignore_index=True)

    post_data.to_csv('../../post_data.csv')